# Tensorflow operations

In [2]:
from qs_mps.mps_class import MPS
from qs_mps.utils import get_precision, tensor_shapes
import numpy as np

2024-01-29 15:57:56.266539: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
h_i = 2
h_f = 6
npoints = 20
interval = np.linspace(h_i,h_f,npoints)
num = (h_f - h_i) / npoints
precision = get_precision(num)
L=6
l=5
d=2**l
model="Z2_dual"
chi=64
charges_x = None
charges_y = None
path_tensor = "/Users/fradm98/Desktop/projects/1_Z2"
# for h in interval:
h = 2.0
lattice_mps = MPS(L=L, d=d, model=model, chi=chi, h=h)
lattice_mps.L = lattice_mps.L - 1

lattice_mps.load_sites(path=path_tensor, precision=precision, cx=charges_x, cy=charges_y)
lattice_mps.Z2.mpo_Z2_ladder_generalized()
lattice_mps.w = lattice_mps.Z2.mpo.copy()

In [4]:
tensor_shapes(lattice_mps.sites)

(1, 32, 32)
(32, 32, 64)
(64, 32, 64)
(64, 32, 32)
(32, 32, 1)


[(1, 32, 32), (32, 32, 64), (64, 32, 64), (64, 32, 32), (32, 32, 1)]

In [5]:
from ncon import ncon
import tensorflow as tf

## Contration with `ncon`

We execute the contraction, with the function `ncon`, of the environments to get the $H_{eff}$.  
Subsequently, we reshape the tensor for the diagonalization of the effective matrix.

In [6]:
a = np.array([1]) 
v_l = np.zeros((lattice_mps.w[0].shape[0]))
v_l[0] = 1
env_l = ncon([a,v_l,a],[[-1],[-2],[-3]])
env_l = ncon([env_l, lattice_mps.sites[0], lattice_mps.w[0], lattice_mps.sites[0].conjugate()],[[1,2,3],[1,4,-1],[2,-2,4,5],[3,5,-3]])
v_r = np.zeros((lattice_mps.w[-1].shape[1]))
v_r[-1] = 1
env_r = ncon([a,v_r.T,a],[[-1],[-2],[-3]])
env_r = ncon([env_r, lattice_mps.sites[-1], lattice_mps.w[-1], lattice_mps.sites[-1].conjugate()],[[1,2,3],[-1,4,1],[-2,2,4,5],[-3,5,3]])
print("Here")
arr = ncon([env_l,lattice_mps.w[1],env_r],[[-1,1,-4],[1,2,-2,-5],[-3,2,-6]])
%timeit ncon([env_l,lattice_mps.w[1],env_r],[[-1,1,-4],[1,2,-2,-5],[-3,2,-6]])
print(arr.shape)

Here
9.61 s ± 134 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
(32, 32, 32, 32, 32, 32)


In [25]:
# reshape
%timeit arr.reshape((2**15,2**15))
arr_resh = arr.reshape((2**15,2**15))

12.1 s ± 118 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The results show a total time of $22.41 s$ for contraction and reshaping.

## Contraction with `tf.tensordot()`

The same contraction and reshaping can be executed by converting the `numpy.ndarray`s in `tensorflow`'s objects

In [8]:
mpo_mid = tf.constant(lattice_mps.w[1])
env_l = tf.constant(env_l)
env_r = tf.constant(env_r)
print(mpo_mid.get_shape(), env_l.get_shape(), env_r.get_shape())

(7, 7, 32, 32) (32, 7, 32) (32, 7, 32)


In [9]:
env = tf.tensordot(env_l,mpo_mid,[[1],[0]])
%timeit tf.tensordot(env,env_r,[[2],[1]])
env = tf.tensordot(env,env_r,[[2],[1]])

3.56 s ± 39.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit env_new = tf.reshape(env, (2**15,2**15))
env_new = tf.reshape(env, (2**15,2**15))
print(env_new.get_shape())

72.5 µs ± 435 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
(32768, 32768)


In [22]:
print(env_new.get_shape())
%timeit b = env_new.numpy()

(32768, 32768)
8.63 s ± 92.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The total time of contraction and reshaping is faster by a factor $\text{x}6$. The only problem is that to execute the diagonalization, we need to get back the `numpy.ndarray` and perform the sparse operation `eigsh`. This conversion takes $8.63s$, the advantage is still present in the case we use tensorflow

In [26]:
from scipy.sparse.linalg import eigsh
%timeit e,v = eigsh(env_new.numpy(), k=1, which="SA")
%timeit e,v = eigsh(arr_resh, k=1, which="SA")

15.9 s ± 265 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
28.1 s ± 2.58 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
